# TME 1

## 1. Needleman & Wunsch

In [1]:
import numpy as np

def base_distance_matrix(alphabet: list[str], *, match_score: float, mismatch_score: float):
  return np.ones((len(alphabet), len(alphabet)), dtype=int) * mismatch_score + (match_score - mismatch_score) * np.identity(len(alphabet), dtype=int)

base_distance_matrix(['A', 'C', 'G', 'T'], match_score=1, mismatch_score=-1)


array([[ 1, -1, -1, -1],
       [-1,  1, -1, -1],
       [-1, -1,  1, -1],
       [-1, -1, -1,  1]])

In [2]:
from typing import Optional, Sequence

def pack(values: Sequence[bool]):
  return sum((1 << i) if values[i] else 0 for i in range(len(values)))

def align_nw(
    seq1: Sequence[str],
    seq2: Sequence[str],
    *,
    alphabet: Optional[list[str]] = None,
    distance_matrix: Optional[np.ndarray] = None,
    gap_score_open: int = -2,
    gap_score_extension: int = -1
):
  alphabet = list(set(seq1) | set(seq2)) if alphabet is None else alphabet
  distance_matrix = base_distance_matrix(alphabet, match_score=1, mismatch_score=-2) if distance_matrix is None else distance_matrix

  seq1_n = [alphabet.index(x) for x in seq1]
  seq2_n = [alphabet.index(x) for x in seq2]

  # Score matrix construction

  score = np.zeros((len(seq1) + 1, len(seq2) + 1))
  arrows= np.zeros((len(seq1) + 1, len(seq2) + 1), dtype=np.uint8)

  score[0, :] = np.arange(0, -score.shape[1], step=-1)
  score[:, 0] = np.arange(0, -score.shape[0], step=-1)

  arrows[0, 1:] = 1
  arrows[1:, 0] = 2

  for index1, letter1 in enumerate(seq1_n, start=1):
    for index2, letter2 in enumerate(seq2_n, start=1):
      match_score = distance_matrix[letter1, letter2]

      a = score[index1, index2 - 1] + (gap_score_extension if (arrows[index1, index2 - 1] & 1) != 0 else gap_score_open)
      b = score[index1 - 1, index2] + (gap_score_extension if (arrows[index1 - 1, index2] & 2) != 0 else gap_score_open)
      c = score[index1 - 1, index2 - 1] + match_score
      m = max(a, b, c)

      score[index1, index2] = m
      arrows[index1, index2] = pack([a == m, b == m, c == m])

  # Traceback

  position = (len(seq1), len(seq2))
  seq1_a = list[str]()
  seq2_a = list[str]()

  while position != (0, 0):
    if (arrows[position] & 1) != 0:
      seq1_a.append('-')
      seq2_a.append(seq2[position[1] - 1])
      position = (position[0], position[1] - 1)
    elif (arrows[position] & 2) != 0:
      seq1_a.append(seq1[position[0] - 1])
      seq2_a.append('-')
      position = (position[0] - 1, position[1])
    elif (arrows[position] & 4) != 0:
      seq1_a.append(seq1[position[0] - 1])
      seq2_a.append(seq2[position[1] - 1])
      position = (position[0] - 1, position[1] - 1)
    else:
      raise RuntimeError

  return (
    str().join(list(reversed(seq1_a))),
    str().join(list(reversed(seq2_a)))
  )

align_nw('TCTGAAC', 'CATGAC')


('TC-TGAAC', '-CATGA-C')

The algorithm's complexity is $O(n^2)$ as we're iterating the sequence on two dimensions to create the matrix.

The alignment is not necessarily unique as there can be multiple paths to reach the top-left cell from the bottom-right cell.

## 2. A small example

In [3]:
# Load the BLOSUM62 matrix

from io import StringIO

blosum62_file = StringIO('''A R N D C Q E G H I L K M F P S T W Y V B J Z X *
4 -1 -2 -2 0 -1 -1 0 -2 -1 -1 -1 -1 -2 -1 1 0 -3 -2 0 -2 -1 -1 -1 -4
-1 5 0 -2 -3 1 0 -2 0 -3 -2 2 -1 -3 -2 -1 -1 -3 -2 -3 -1 -2 0 -1 -4
-2 0 6 1 -3 0 0 0 1 -3 -3 0 -2 -3 -2 1 0 -4 -2 -3 4 -3 0 -1 -4
-2 -2 1 6 -3 0 2 -1 -1 -3 -4 -1 -3 -3 -1 0 -1 -4 -3 -3 4 -3 1 -1 -4
0 -3 -3 -3 9 -3 -4 -3 -3 -1 -1 -3 -1 -2 -3 -1 -1 -2 -2 -1 -3 -1 -3 -1 -4
-1 1 0 0 -3 5 2 -2 0 -3 -2 1 0 -3 -1 0 -1 -2 -1 -2 0 -2 4 -1 -4
-1 0 0 2 -4 2 5 -2 0 -3 -3 1 -2 -3 -1 0 -1 -3 -2 -2 1 -3 4 -1 -4
0 -2 0 -1 -3 -2 -2 6 -2 -4 -4 -2 -3 -3 -2 0 -2 -2 -3 -3 -1 -4 -2 -1 -4
-2 0 1 -1 -3 0 0 -2 8 -3 -3 -1 -2 -1 -2 -1 -2 -2 2 -3 0 -3 0 -1 -4
-1 -3 -3 -3 -1 -3 -3 -4 -3 4 2 -3 1 0 -3 -2 -1 -3 -1 3 -3 3 -3 -1 -4
-1 -2 -3 -4 -1 -2 -3 -4 -3 2 4 -2 2 0 -3 -2 -1 -2 -1 1 -4 3 -3 -1 -4
-1 2 0 -1 -3 1 1 -2 -1 -3 -2 5 -1 -3 -1 0 -1 -3 -2 -2 0 -3 1 -1 -4
-1 -1 -2 -3 -1 0 -2 -3 -2 1 2 -1 5 0 -2 -1 -1 -1 -1 1 -3 2 -1 -1 -4
-2 -3 -3 -3 -2 -3 -3 -3 -1 0 0 -3 0 6 -4 -2 -2 1 3 -1 -3 0 -3 -1 -4
-1 -2 -2 -1 -3 -1 -1 -2 -2 -3 -3 -1 -2 -4 7 -1 -1 -4 -3 -2 -2 -3 -1 -1 -4
1 -1 1 0 -1 0 0 0 -1 -2 -2 0 -1 -2 -1 4 1 -3 -2 -2 0 -2 0 -1 -4
0 -1 0 -1 -1 -1 -1 -2 -2 -1 -1 -1 -1 -2 -1 1 5 -2 -2 0 -1 -1 -1 -1 -4
-3 -3 -4 -4 -2 -2 -3 -2 -2 -3 -2 -3 -1 1 -4 -3 -2 11 2 -3 -4 -2 -2 -1 -4
-2 -2 -2 -3 -2 -1 -2 -3 2 -1 -1 -2 -1 3 -3 -2 -2 2 7 -1 -3 -1 -2 -1 -4
0 -3 -3 -3 -1 -2 -2 -3 -3 3 1 -2 1 -1 -2 -2 0 -3 -1 4 -3 2 -2 -1 -4
-2 -1 4 4 -3 0 1 -1 0 -3 -4 0 -3 -3 -2 0 -1 -4 -3 -3 4 -3 0 -1 -4
-1 -2 -3 -3 -1 -2 -3 -4 -3 3 3 -3 2 0 -3 -2 -1 -2 -1 2 -3 3 -3 -1 -4
-1 0 0 1 -3 4 4 -2 0 -3 -3 1 -1 -3 -1 0 -1 -2 -2 -2 0 -3 4 -1 -4
-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -4
-4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 1
''')

blosum62_alphabet = blosum62_file.readline()[0:-1].split(' ')
blosum62_matrix = np.loadtxt(blosum62_file, delimiter=' ', dtype=int)


In [4]:
# Fetch sequences from rcsb.org

import urllib.request

def fetch_rcsb_sequence(name: str):
  response = urllib.request.urlopen(f'https://www.rcsb.org/fasta/entry/{name}')

  while raw_line := response.readline().decode():
    line = raw_line.rstrip()

    if not line.startswith('>'):
      return line

  raise RuntimeError('No sequence found')

seq_2abl = fetch_rcsb_sequence('2ABL')
seq_1opk = fetch_rcsb_sequence('1OPK')

print(seq_2abl)
print(seq_1opk)


MGPSENDPNLFVALYDFVASGDNTLSITKGEKLRVLGYNHNGEWCEAQTKNGQGWVPSNYITPVNSLEKHSWYHGPVSRNAAEYLLSSGINGSFLVRESESSPGQRSISLRYEGRVYHYRINTASDGKLYVSSESRFNTLAELVHHHSTVADGLITTLHYPAP
GAMDPSEALQRPVASDFEPQGLSEAARWNSKENLLAGPSENDPNLFVALYDFVASGDNTLSITKGEKLRVLGYNHNGEWCEAQTKNGQGWVPSNYITPVNSLEKHSWYHGPVSRNAAEYLLSSGINGSFLVRESESSPGQRSISLRYEGRVYHYRINTASDGKLYVSSESRFNTLAELVHHHSTVADGLITTLHYPAPKRNKPTIYGVSPNYDKWEMERTDITMKHKLGGGQYGEVYEGVWKKYSLTVAVKTLKEDTMEVEEFLKEAAVMKEIKHPNLVQLLGVCTREPPFYIITEFMTYGNLLDYLRECNRQEVSAVVLLYMATQISSAMEYLEKKNFIHRNLAARNCLVGENHLVKVADFGLSRLMTGDTYTAHAGAKFPIKWTAPESLAYNKFSIKSDVWAFGVLLWEIATYGMSPYPGIDLSQVYELLEKDYRMERPEGCPEKVYELMRACWQWNPSDRPSFAEIHQAFETMFQESSISDEVEKELGKRGT


In [5]:
# Align sequences

alignment = align_nw(
  seq_2abl + '*',
  seq_1opk + '*',
  alphabet=blosum62_alphabet,
  distance_matrix=blosum62_matrix,
  gap_score_extension=-1,
  gap_score_open=-11
)

print(alignment)


('-----------------------------------MGPSENDPNLFVALYDFVASGDNTLSITKGEKLRVLGYNHNGEWCEAQTKNGQGWVPSNYITPVNSLEKHSWYHGPVSRNAAEYLLSSGINGSFLVRESESSPGQRSISLRYEGRVYHYRINTASDGKLYVSSESRFNTLAELVHHHSTVADGLITTLHYPAP---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------*', 'GAMDPSEALQRPVASDFEPQGLSEAARWNSKENLLAGPSENDPNLFVALYDFVASGDNTLSITKGEKLRVLGYNHNGEWCEAQTKNGQGWVPSNYITPVNSLEKHSWYHGPVSRNAAEYLLSSGINGSFLVRESESSPGQRSISLRYEGRVYHYRINTASDGKLYVSSESRFNTLAELVHHHSTVADGLITTLHYPAPKRNKPTIYGVSPNYDKWEMERTDITMKHKLGGGQYGEVYEGVWKKYSLTVAVKTLKEDTMEVEEFLKEAAVMKEIKHPNLVQLLGVCTREPPFYIITEFMTYGNLLDYLRECNRQEVSAVVLLYMATQISSAMEYLEKKNFIHRNLAARNCLVGENHLVKVADFGLSRLMTGDTYTAHAGAKFPIKWTAPESLAYNKFSIKSDVWAFGVLLWEIATYGMSPYPGIDLSQVYELLEKDYRMERPEGCPEKVYELMRACWQWNPSDRPSFAEIHQAFETMFQESSISDEVEKELGKRGT*')

In [6]:
# Improve formatting

from typing import TypeVar

T = TypeVar('T', str, list)

def partition(items: T, size: int) -> list[T]:
  return [items[index:(index + size)] for index in range(0, len(items), size)]

def format_alignment(seq1: str, seq2: str, label1: str, label2: str, *, width: int = 120):
  label_size = max(len(label1), len(label2))
  format_match = lambda a, b: ('|' if a == b else ':') if (a != '-' and b != '-') else ' '

  return '\n\n\n'.join([
    f'''{label1:{label_size}} {line1}\n{' ' * label_size} {str().join(format_match(x, y) for x, y in zip(line1, line2))}\n{label2:{label_size}} {line2}''' for line1, line2 in zip(partition(seq1, width), partition(seq2, width))
  ])

print(format_alignment(*alignment, '2ABL', '1OPK'))


2ABL -----------------------------------MGPSENDPNLFVALYDFVASGDNTLSITKGEKLRVLGYNHNGEWCEAQTKNGQGWVPSNYITPVNSLEKHSWYHGPVSRNAAEYL
                                        :||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
1OPK GAMDPSEALQRPVASDFEPQGLSEAARWNSKENLLAGPSENDPNLFVALYDFVASGDNTLSITKGEKLRVLGYNHNGEWCEAQTKNGQGWVPSNYITPVNSLEKHSWYHGPVSRNAAEYL


2ABL LSSGINGSFLVRESESSPGQRSISLRYEGRVYHYRINTASDGKLYVSSESRFNTLAELVHHHSTVADGLITTLHYPAP------------------------------------------
     ||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||                                          
1OPK LSSGINGSFLVRESESSPGQRSISLRYEGRVYHYRINTASDGKLYVSSESRFNTLAELVHHHSTVADGLITTLHYPAPKRNKPTIYGVSPNYDKWEMERTDITMKHKLGGGQYGEVYEGV


2ABL ------------------------------------------------------------------------------------------------------------------------
                                                                                                                  

Using BLAST (more precisely blastp), we get:

```
2ABL   2    GPSENDPNLFVALYDFVASGDNTLSITKGEKLRVLGYNHNGEWCEAQTKNGQGWVPSNYI  61
            GPSENDPNLFVALYDFVASGDNTLSITKGEKLRVLGYNHNGEWCEAQTKNGQGWVPSNYI
1OPK   37   GPSENDPNLFVALYDFVASGDNTLSITKGEKLRVLGYNHNGEWCEAQTKNGQGWVPSNYI  96

2ABL   62   TPVNSLEKHSWYHGPVSRNAAEYLLSSGINGSFLVRESESSPGQRSISLRYEGRVYHYRI  121
            TPVNSLEKHSWYHGPVSRNAAEYLLSSGINGSFLVRESESSPGQRSISLRYEGRVYHYRI
1OPK   97   TPVNSLEKHSWYHGPVSRNAAEYLLSSGINGSFLVRESESSPGQRSISLRYEGRVYHYRI  156

2ABL   122  NTASDGKLYVSSESRFNTLAELVHHHSTVADGLITTLHYPAP  163
            NTASDGKLYVSSESRFNTLAELVHHHSTVADGLITTLHYPAP
1OPK   157  NTASDGKLYVSSESRFNTLAELVHHHSTVADGLITTLHYPAP  198
```

which is exactly the same.

When comparing structures, we see that despite having the same sequence and secondary structures, the tertiary structures of 2ABL and 1OPK do not align very well.

The BLAST algorithm uses a heuristic that approximates the Smith-Waterman algorithm but that is much faster.

In [16]:
from pathlib import Path
from typing import Optional, Sequence, TypeVar
import urllib.request
import numpy as np

def base_distance_matrix(alphabet: list[str], *, match_score: float, mismatch_score: float):
    return np.ones((len(alphabet), len(alphabet)), dtype=int) * mismatch_score + (match_score - mismatch_score) * np.identity(len(alphabet), dtype=int)

def _fetch_rcsb_sequence(name: str):
    response = urllib.request.urlopen(f'https://www.rcsb.org/fasta/entry/{name}')

    while raw_line := response.readline().decode():
        line = raw_line.rstrip()

        if not line.startswith('>'):
            return line

    raise RuntimeError('No sequence found')

def _pack(values: Sequence[bool]):
    return sum((1 << i) if values[i] else 0 for i in range(len(values)))

T = TypeVar('T', str, list)

def _partition(items: T, size: int) -> list[T]:
    return [items[index:(index + size)] for index in range(0, len(items), size)]

def align_nw(seq1: Sequence[str], seq2: Sequence[str], *,
             alphabet: Optional[list[str]] = None,
             distance_matrix: Optional[np.ndarray] = None,
             gap_score_open: int = -2,
             gap_score_extension: int = -1):
    alphabet = list(set(seq1) | set(seq2)) if alphabet is None else alphabet
    distance_matrix = base_distance_matrix(alphabet, match_score=1, mismatch_score=-2) if distance_matrix is None else distance_matrix

    seq1_n = [alphabet.index(x) for x in seq1]
    seq2_n = [alphabet.index(x) for x in seq2]

    # Score matrix construction
    score = np.zeros((len(seq1) + 1, len(seq2) + 1))
    arrows = np.zeros((len(seq1) + 1, len(seq2) + 1), dtype=np.uint8)

    score[0, :] = np.arange(0, -score.shape[1], step=-1)
    score[:, 0] = np.arange(0, -score.shape[0], step=-1)

    arrows[0, 1:] = 1
    arrows[1:, 0] = 2

    max_score = 0  # Track the maximum score
    max_position = (0, 0)  # Track the position of the maximum score

    for index1, letter1 in enumerate(seq1_n, start=1):
        for index2, letter2 in enumerate(seq2_n, start=1):
            match_score = distance_matrix[letter1, letter2]

            a = score[index1, index2 - 1] + (gap_score_extension if (arrows[index1, index2 - 1] & 1) != 0 else gap_score_open)
            b = score[index1 - 1, index2] + (gap_score_extension if (arrows[index1 - 1, index2] & 2) != 0 else gap_score_open)
            c = score[index1 - 1, index2 - 1] + match_score
            m = max(a, b, c, 0)  # Ensure no negative scores

            score[index1, index2] = m
            arrows[index1, index2] = _pack([a == m, b == m, c == m])

            # Update the maximum score and its position
            if m > max_score:
                max_score = m
                max_position = (index1, index2)

    # Traceback for local alignment
    position = max_position
    seq1_a = list[str]()
    seq2_a = list[str]()

    while score[position] > 0:
        if (arrows[position] & 1) != 0:
            seq1_a.append('-')
            seq2_a.append(seq2[position[1] - 1])
            position = (position[0], position[1] - 1)
        elif (arrows[position] & 2) != 0:
            seq1_a.append(seq1[position[0] - 1])
            seq2_a.append('-')
            position = (position[0] - 1, position[1])
        elif (arrows[position] & 4) != 0:
            seq1_a.append(seq1[position[0] - 1])
            seq2_a.append(seq2[position[1] - 1])
            position = (position[0] - 1, position[1] - 1)
        else:
            raise RuntimeError

    # Reverse the sequences
    seq1_a = seq1_a[::-1]
    seq2_a = seq2_a[::-1]

    return str().join(seq1_a), str().join(seq2_a),max_position

def format_alignment(seq1: str, seq2: str, label1: str, label2: str, *, width: int = 80):
    label_size = max(len(label1), len(label2)
                     )
    format_match = lambda a, b: ('|' if a == b else ':') if (a != '-' and b != '-') else ' '

    return '\n\n\n'.join([
        f'''{label1:{label_size}} {line1}\n{' ' * label_size} {str().join(format_match(x, y) for x, y in zip(line1, line2))}\n{label2:{label_size}} {line2}'''
        for line1, line2 in zip(_partition(seq1, width), _partition(seq2, width))
    ])

# Load the Blosum62 matrix
with Path('blosum62.txt').open('rt') as file:
    blosum62_alphabet = file.readline()[0:-1].split(' ')
    blosum62_matrix = np.loadtxt(file, delimiter=' ', dtype=int)

# Align sequences and print the formatted result
a, b, position = align_nw(
    list(_fetch_rcsb_sequence('2ABL')),
    list(_fetch_rcsb_sequence('1OPK')),
    alphabet=blosum62_alphabet,
    distance_matrix=blosum62_matrix,
    gap_score_open=-11,
    gap_score_extension=-1
)

print("position on the first sequence:",position[0])
print("position sur la deuxième séquence:",position[1])
print(format_alignment(a, b, '2ABL', '1OPK', width=80))


position on the first sequence: 163
position sur la deuxième séquence: 198
2ABL GPSENDPNLFVALYDFVASGDNTLSITKGEKLRVLGYNHNGEWCEAQTKNGQGWVPSNYITPVNSLEKHSWYHGPVSRNA
     ||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
1OPK GPSENDPNLFVALYDFVASGDNTLSITKGEKLRVLGYNHNGEWCEAQTKNGQGWVPSNYITPVNSLEKHSWYHGPVSRNA


2ABL AEYLLSSGINGSFLVRESESSPGQRSISLRYEGRVYHYRINTASDGKLYVSSESRFNTLAELVHHHSTVADGLITTLHYP
     ||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
1OPK AEYLLSSGINGSFLVRESESSPGQRSISLRYEGRVYHYRINTASDGKLYVSSESRFNTLAELVHHHSTVADGLITTLHYP


2ABL AP
     ||
1OPK AP
